# Python language- Advanced

## Everything in Python is an object-advanced

### Scalar type과 Collection type

http://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/
https://ahracho.github.io/posts/python/2017-05-01-everything-in-python-is-object-integer/

python 제공 기본 타입들은 크게 Scalar type과 Collection type으로 나눠볼 수 있다. 이들 모두가 object일텐데 어떻게 구상되어 있을까? 간단히 생각해보면 공통적으로는 다음의 정보를 가지고 있을테고
- data type: int인지, string인지 등. 파이썬에서는 변수에 데이터 타입을 명시하지 않으니 object안에 그 정보가 저장되어야 함
- reference count : 몇 개의 변수가 나를 사용하고 있는지 알아야 한다. 아무도 나를 사용하지 않는다면 굳이 메모리를 차지하면서 있을 필요가 없을테니까.


다른 점을 생각해보면
Scalar type은
- value: 5라는 정수값, 또는 'aaa'와 같은문자열 값 등 값 자체

Collection type은
- items: 즉 값 자체가 아닌 값들에 대한 주소값을 가진다

### Mutable and immutable objects
- immutable type: Scalar Types전체, str, tuple
- mutable type: list, set, dict, NumPy의 배열(ndarray)
    - mutable하다는게 이걸 권장한다는건 아님


Mutable하다, immutable하다는 말은 앞에서 생각해본 해당 object가 지니고 있는 값이 변할 수 있냐 없냐의 의미이다. 이 object를 가리키고 있는 변수는 당연히 변할 수 있다. 따라서 Scalar인지 collection인지에 따라 그 의미가 조금 다를 수 있다.
- Scalar Type에서는 value가,
- collection type에서는 element의 value(더 정확히 말하면 element id) 변경이 불가하다는 것이다.

In [1]:
# mutable-collection 예시: 반면 list는 element 수정 가능
# a의 id는 유지되고, 수정한 a[0]의 id값만 바뀜
a = [1, 2, 3]; print(id(a), id(a[0]))
a[0] = 4; print(id(a), id(a[0]),a)

4372726800 4332680320
4372726800 4332680416 [4, 2, 3]


In [ ]:
# immutable-collection 예시. 아래와 같이 element를 바꾸려고 하면 오류가 난다.
# element를 바꾸려면 tuple object의 items 값을 바꿔줘야 하는데 그건 바꿀 수 없기 떄문
a = (1, 2, 3)
a[0] = 4

In [2]:
#당연히 tuple안에 있는 list의 items값을 바꾸는것은 가능. a[0]=1은 안되지만
a = ([1,2], 2, 3); print(id(a), id(a[0]))
a[0][1] = 4; print(id(a), id(a[0]),a)

4372504976 4372995440
4372504976 4372995440 ([1, 4], 2, 3)


아래와 같이 변수값은 당연히 바뀔 수 있음

In [32]:
a = 1; a= 2 # int가 immutable임에도 불구하고 값이 바뀌는것 처럼 보이는것은 실제로는 int object가 변하는게 아니라 변수값이 바뀌는 것
b = (1,2); b = (1,2,3)
c = (4,5,6); c+=b

immutable type끼리 연산도 된다. 그런데 자세히 들여다보면 Mutable and immutable type의 연산하는 방법이 조금 다름
- Mutable은 값이 바뀔 수 있으니 기존 object를 그대로 사용할 수 있는데(그래서 id값이 바뀌지 않는데
- immutable은 값이 바뀔 수 없으니 기존 object 대신 새로운 object를 만들고 변수에 이를 새로 할당함

In [23]:
# Mutable에서는 id 값이 그대로
l1 = [1, 2, 3] ; print(id(l1), id(l1[0]))
l2 = [4, 5]   
l1 += l2 ; print(id(l1), id(l1[0]), t1)

#반면 immutable type간 연산: 값이 바뀌는게 아닌 새로운 immutable object를 만들어 가리키던 변수를 옮기는 것
# 단 이 때도 내부 element
t1 = (1, 2, 3) ; print(id(t1), id(t1[0]))
t2 = (4, 5)   
t1 += t2 ; print(id(t1),id(t1[0]), t1)
t1

# 당연히 재할당하면 mutable한 list라도 id 바뀜
a = [1, 2, 3]; print(id(a))
a = [4, 5, 6]; print(id(a))

4348998032 4311045248
4348998032 4311045248 (1, 2, 3, 4, 5)
4350182256 4311045248
4349781104 4311045248 (1, 2, 3, 4, 5)
4350275744
4350735376


이에 따라 function에 argument passing하거나, 값 복사 할 때 주의가 필요함

#### argument passing
- 변수에 값을 할당하면 이는 값이 복사되는게 아닌 해당 변수명(이름)을 객체에 연결하는 것이므로 binding이라 부름
- 전달하는 변수가 mutable인지 아닌지에 따라 변수값이 함수 내부연산결과로 바뀌는지 아닌지가 달라짐
- 그냥 항상 리턴받는걸로 하는게 안전하려나?

In [32]:
#mutable: 전달한 변수가 함께 바뀐다. 리턴받지 않았음에도
def append_element(some_list, element):
    some_list.append(element)
    print(some_list)
l = [1, 2, 3]
append_element(l, 4)
print(l)

#immutable: 전달한 변수는 바뀌지 않는다.
def sum_one(some_int, element):
    some_int+=1
    print(some_int)

i=0
sum_one(i, 1)
print(i)

[1, 2, 3, 4]
[1, 2, 3, 4]
1
0


#### shallow copy와 deep copy: mutable type 복사시 고려사항

따라서 mutable type을 사용하면서 b=a같이 복사할 때는 주의가 필요하다 
https://suwoni-codelab.com/python%20%EA%B8%B0%EB%B3%B8/2018/03/02/Python-Basic-copy/
- immutable은 동일한 객체를 두 변수가 가리키고 있을 떄 한 변수값을 변화시키려고 하면 object가 복사되면서 분리되고, 따라서 한 변수의 변화가 다른 변수에 영향을 끼치지 않지만
- mutable은 한 변수값을 변화시키면 object값이 그대로 변화가 되기에 다른 변수로 접근해도 당연히 변화된 값으로 접근된다. -> shallow copy와 deep copy 고려해야함

In [4]:
# 
a = 1
b = a       # 변수 b에 변수 a를 대입하면
print(a is b)      # 변수 a, b는 같은 객체를 가리키게 됩니다. 

a += 1       # 변수 a 값을 변경하려고 하면 integer는 immutable type이니 이를 위해 새로운 객체를 생성해야만 하고
print(a is b) # 결과적으로 변수 a와 b는 별개의 객체를 가리키게 됩니다. 
print(a,b)    # 당연히 변수 a와 b가 가리키는 객체의 값이 다릅니다. 

True
False
3 1


In [24]:
# mutable 
a = [1, 2, 3]
b = a           # 변수 b에 변수 a를 대입하면
print(a is b)   # 변수 a, b는 같은 객체를 가리키게 됩니다. 

a.append(4)     # 변수 a값을 변경해도 list는 mutable type이라 값이 변경됩니다.
print(a is b)   # 그 결과 변수 a와 b는 같은 객체를 가리키고 있습니다. 
print(a,b)      # 변수 a만 변경했음에도 b도 같이 변한것 처럼 보이게됨(동일 객체를 가리키니) 

True
True
[1, 2, 3, 4] [1, 2, 3, 4]


##### 대입 VS shallow/deep copy

동일한 값을 가진 새로운 객체를 만들고 싶을 때는 대입말고 copy.copy() 또는 copy.deepcopy()를 이용해야함

In [33]:
import copy

a = [ 1, 2, 3]
b = a      # 대입
c = copy.copy(a) # 얕은 복사 
d = copy.deepcopy(a) # 깊은 복사 

print(a, b, c, d) # 모두 같은 값을 가지고 있지만
print(a is b , a is c , a is d ) # 변수 와 b만(대입) 같은 객체를 가리킵니다.  

b[0]=100  # 변수 b가 가리키는 리스트 객체 원소를 수정했더니 a만 함께 수정됨
print(a, b, c, d)
# 여기서는 얕은 복사와 깊은 복사간에 차이가 없습니다. 

[1, 2, 3] [1, 2, 3] [1, 2, 3] [1, 2, 3]
True False False
[100, 2, 3] [100, 2, 3] [1, 2, 3] [1, 2, 3]


##### shallow VS deep copy

그러나 리스트 객체의 원소가 immutable 객체가 아닌 mutable 객체(list, set, dictionary등)인 경우에는 얕은 복사(shallow copy)와 깊은 복사(deep copy)간에 차이가 발생하니 주의
즉 list 등을 원소로 하는 list를 복사할 때는 deepcopy이용해야 한다

In [39]:
import copy
a = [['a', 'b'], [1, 2]]
b = copy.copy(a)
c = copy.deepcopy(a)

print(a is b , a is c)   # 변수 b,c모두는 a와 다른 별개의 객체를 가리키고 있지만 
# list 안 list는 copy()의 경우, 기존과 같은 객체를 가리키고 있습니다. 
# 반면 deepcopy()의 경우, 새로운 객체를 생성해서 다른 객체를 가리키고 있습니다.
print(a[0] is b[0] , a[0] is c[0])  
a[0].append('c')  # 그래서 한쪽 리스트 객체의 원소가 가리키는 객체의 값을 변경하면 
print(a, b, c)# copy로 만든 객체에는 반영 됩니다. 

False False
True False
[['a', 'b', 'c'], [1, 2]] [['a', 'b', 'c'], [1, 2]] [['a', 'b'], [1, 2]]


# Pythonic(파이썬을 좀 더 파이썬스럽게)

## Python coding style

- Python Enhancement Proposal 8 (PEP 8): https://www.python.org/dev/peps/pep-0008/
- PEP8 한글 번역:
https://kongdols-room.tistory.com/18

### Code layout

#### Comments

In [27]:
# 이렇게 한 칸 띄고 시작하기.  앞 처럼 문장 끝에는 스페이스 두개 추가하기
a = 1 + 1  #구문으로부터 최소 2개 space 띄우기

In [28]:
"""
원래 파이썬이네느 여러줄 주석이란 개념은 없지만 """ """ 를 사용할 수 있음
(원래는 " " 사이에 문자열을 저장 하기 위해 사용되지만 앞에 별다른 변수가 없다면 무시가됩니다)
"""

'\n원래 파이썬이네느 여러줄 주석이란 개념은 없지만  를 사용할 수 있음\n(원래는 " " 사이에 문자열을 저장 하기 위해 사용되지만 앞에 별다른 변수가 없다면 무시가됩니다)\n'

#### Docstrings

#### Indentation, 줄바꿈
- Tab 대신 4개의 공백(Space) 사용
- 라인이 길어 줄바꿈해야 할 때는, 괄호위치에 맞춰주거나, 최소한 괄호 밖의 다른 요소와 구분될 수 있게 추가 indetation해줘야 함

#### Whitespace
쓸데없는 공백 주지말기

기본적으로는 연산자 주변에 공백을 두되, 우선순위에 따라 공백 제거해서 쓰는것도 고려

In [29]:
var = 0; a = 0; b = 0; c = 0

var = var + 1  #Yes
var=var+1  #No

var += 1  #Yes
var +=1  #No

var = var*2 - 1  #Yes
var = var * 2 - 1  #No

var = var*var + var*var  #Yes
var = var * var + var * var  #No

c = (a+b) * (a-b)  #Yes
c = (a + b) * (a - b)  #No

# 단 =가 대입연산자가 아닌 argument나 default value로 쓰일경우, 공백 넣지 않기
def complex(real, imag=0.0):
    return magic(r=real, i=imag)

#### import문
- 항상 파일의 최상단에 넣기
- Standard library > 3rd Party 모듈 > 자신의 모듈 순으로
- 한 라인에 하나의 모듈: import os, sys(x)

In [30]:
import os
import sys
from subprocess import Popen, PIPE # 이건 한 모듈 안에 여러개니 ok

### naming

- 클래스는 CoreClass
- 패키지명은 totalnumbers(소문자를 사용하지만 밑줄은 사용하지 않음)
- 모듈, 함수, 변수, Attribute 명은 total_numbers
- 모듈 상수 MAX_COUNT
- class의 attrubute/method
    - public attribute: 밑줄없이 name
    - protected instance attribute는 밑줄 하나로 시작 _initialized
    - private instance attribute는 밑줄 2개로 시작 __private_var
- method
    - 인스턴스 메서드는 (객체 자신을 가리키기 위해) self 사용	예: def copy(self, other):
    - 클래스 메서드는 (클래스 자신을 가리키기 위해) cls 사용  	예: def clone(cls, other):

### 기타

collection에서 값이 비어있는지 아닌지를 검사하기 위해 길이를 체크하는 방식을 사용하지 말 것. 
- 비어있는 collection은 False임을 이용하여, if mylist 와 같이 표현함

In [31]:
mylist = [1,2,3]

#값이 비어있는지 확인할 때
if len(mylist) == 0: pass #비권장
if not mylist: pass  #권장

if len(mylist) > 0: pass #비권장
if mylist: pass  #권장

슬래시
- 경로는 맥에서 '/', 윈도우에서 '\\'
- 특수문자는 앞에\ 붙여줌: \n: 줄바꿈, \t: 탭 문자, \\\: \ 문자 자체를 출력
- 여러줄 쓸 때 뒤에 '\\' (괄호로 묶어져있으면 안써도 됨)

In [32]:
special_character = '\n'
path = 'data/titanic_train.csv'

In [33]:
if a == True and \
b == False :
    pass

변수 한번에 할당하기

In [34]:
#이게 사실은 오른쪽 세 요소를 가진 tuple이 unpacking되어 왼쪽 각각에 들어간 것
a, b, c = 10, 20, 30

문장 끝을 ;로 끝낼 필요는 없지만 한줄에 여러 명렬어를 구분하기 위해 쓰기도 함

In [35]:
a=1 ; b=2 ; c=3

### asterisk(*) 사용 용도

파이썬에서 **Asterisk(*)**는 곱셈 및 거듭제곱 연산 이외에도 다음과 같은 상황에서 사용된다.

- 리스트형 컨테이너 타입의 데이터를 반복 확장하고자 할 때 
- 컨테이너 타입의 데이터를 Unpacking 할 때
- 가변인자 (Variadic Arguments)를 사용하고자 할 때

https://hwiyong.tistory.com/193

#### 리스트형 컨테이너 타입의 데이터를 반복 확장하고자 할 때 

In [50]:
# 길이 100의 제로값 리스트 초기화
zeros_list = [0] * 100

# list에 대해 연산하면, 해당 리스트 안에 현재 값들을 여러번 반복해서 넣어줌.
vector_list2 = [1, 2, 3]
print(vector_list2 * 3)

# 그래서 다음과 같이 응용할 수 있음. [[]] 이렇게 써주는 것에 유의!!
vector_list = [[1, 2, 3]] 
print(vector_list * 3)

for i, vector in enumerate(vector_list * 3):
    print("{0} scalar product of vector: {1}".format((i + 1), [(i + 1) * e for e in vector]))

[1, 2, 3, 1, 2, 3, 1, 2, 3]
[[1, 2, 3], [1, 2, 3], [1, 2, 3]]
1 scalar product of vector: [1, 2, 3]
2 scalar product of vector: [2, 4, 6]
3 scalar product of vector: [3, 6, 9]


### 매개변수(parameter)와 전달인자(argument)에 대한 이해

#### 정의
- x, y, z는 매개변수(parameter). 이름에서 볼 수 있듯이 variable임
- 6, 4, 1은 전달인자(argument) 라고함. 값(value)임

In [52]:
def my_function(x, y, z=1.5): 
    pass
    
my_function(6, 4, 1)

#### positional arguments와 keyword argument에 대한 이해
이 둘은 함수를 call할 때, 인자를 전달하는 방식이다. 
- positional arguments: 일반적으로 인자를 넣어주는. 그럼 함수는 순서대로 각 인자를 변수에 대응시킨다.
- keyword arguments: keyword=value 형태로 인자를 넣어주는 방식

참고로 이를 함수를 선언할 때 인자 방식, 즉 defualt값을 지정하는 것(예를 들어 def func(a=None)과    
혼동하는 경우가 있는데 이 두 개는 다르다.   
`arguments`라는 이름에서도 알 수 있듯이(parameter가 아님) 함수를 call 할 때 value를 넣어주는 방식을 의미함

예를 들어 def func(a,b)로 선언한 경우에도 func(b=1,a=1)와 같은 keyword arguments 방식으로 인자 전달이 가능하다)

https://velog.io/@devmin/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%95%A8%EC%88%98%EC%9D%98-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98%EC%9D%98-%ED%8A%B9%EC%A7%95-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0

https://suwoni-codelab.com/python%20%EA%B8%B0%EB%B3%B8/2018/03/05/Python-Basic-argument/

In [34]:
def student_info(name, sex, age) :
  print(f"""
      이름 : {name}
      성별 : {sex}
      나이 : {age}
      """)

student_info("고길동", "남자", 50)  # positional arguments: 
student_info(age = 50, name = "고길동", sex = "남자")  # keyword arguments:


  이름 : 고길동
  성별 : 남자
  나이 : 50
  

  이름 : 고길동
  성별 : 남자
  나이 : 50
  


두 방식의 특성상 아래와 같이 keyword arguments 선언은 무조건 positional arguments 뒤에 와야한다.

In [36]:
student_info("고길동", "남자", age = 50) 


  이름 : 고길동
  성별 : 남자
  나이 : 50
  


In [35]:
student_info(name = "고길동", "남자", 50) 

SyntaxError: positional argument follows keyword argument (<ipython-input-35-9c1c2f57d28b>, line 1)

참고로 함수 선언 시, default 값을 넣어주는 인자들은 유사한 이유로 뒤에 위치해 있어야 한다.   
즉 함수 선언 할 때 default 값 지정할 때나, 함수 call 할 때 keyword 지정해 주는건 모두 맨 뒤에 있어야 함

In [38]:
def student_info(name, sex, age=10) :
  print(f"""
      이름 : {name}
      성별 : {sex}
      나이 : {age}
      """)

In [37]:
def student_info(name, sex='여자', age) :
  print(f"""
      이름 : {name}
      성별 : {sex}
      나이 : {age}
      """)

SyntaxError: non-default argument follows default argument (<ipython-input-37-e5ad7a4f7f5c>, line 1)

#### argument unpacking, parameter packing
우리는 종종 들어오는 인자의 갯수를 모른다거나, 그 어떤 인자라도 모두 받아서 처리를 해야하는때가 있다.  대표적인 예가 print() 함수   
이 때 생각해볼 수 있는 방법은 크게 두 가지다.
- argument unpacking: 함수에 argument를 넣어줄 때 뭔가 표시를 해주거나
- parameter packing: 함수 선언 시, parameter에 뭔가 표시를 해주거나(가변인자)

##### argument unpacking
positional 컨테이너 타입의 데이터들을 unpacking해서 전달하기

In [1]:
def func(a,b,c):
    return a+b+c
func(0,1,2)

3

In [4]:
my_list = [0,1,2]
print( func(*my_list) ) # positional argument

my_dict = {'a':0, 'b':1, 'c':2}
print( func(**my_dict) )  # keyword argument

3
3


##### parameter packing: 함수에서 가변인자(Variadic Parameters) 사용하기

In [28]:
# Variadic positional arguments: tuple로 인자가 들어감
def save_ranking(*args):
    print(args)
save_ranking('ming', 'alice', 'tom', 'wilson', 'roy')

# Variadic keyword arguments: dict로 인자가 들어감
def save_ranking(**kwargs):
    print(kwargs)
save_ranking(first='ming', second='alice', fourth='wilson', third='tom', fifth='roy')

# positional arguments와 keyword arguments를 모두 받을 때
def save_ranking(*args, **kwargs):
    print(args)
    print(kwargs)
save_ranking('ming', 'alice', 'tom', fourth='wilson', fifth='roy')

('ming', 'alice', 'tom', 'wilson', 'roy')
{'first': 'ming', 'second': 'alice', 'fourth': 'wilson', 'third': 'tom', 'fifth': 'roy'}
('ming', 'alice', 'tom')
{'fourth': 'wilson', 'fifth': 'roy'}


모아서 생각해보자!

In [41]:
def last_fimc(name, email, addr=None, age=20, *args, **kwargs):
    print("name: ", name )
    print("email: ", email )
    print("age: ", age )
    
    if addr is None:
        addr = '서울시'
    
    for arg in args:
        print(arg)
        
    for key in kwargs:
        print(key, kwargs[key])

In [45]:
last_fimc('김한길', 'uoneway@gmail.com', 'Seoul', 40, 1, 2, 3, is_Korean=True, birth='0305')
# last_fimc('김한길', 'uoneway@gmail.com', 'Seoul', age=40, 1, 2, 3, is_Korean=True, birth='0305') 
# 위와같이 하면 오류남. 함수 콜 할때, keyword방식이 뒤에 1,2,3 앞에 와서

name:  김한길
email:  uoneway@gmail.com
age:  40
1
2
3
is_Korean True
birth 0305


사실 아래 func1, func2는 거의 효과가 동일하다. 하지만 가독성을 위해서 첫번째 방식을...??

In [ ]:
def func1(*args, **kwargs):
    pass

func1(1.2.3, 'name': '김한길', 'email': 'uoneway@gmail.com')

def func2(param_list=[], param_dict={}, **kwargs):
    pass

func2([1.2.3], {'name': '김한길', 'email': 'uoneway@gmail.com'})

## lambda 함수 활용
파이썬에서 "lambda" 는 런타임에 생성해서 사용할 수 있는 익명 함수로, 코드의 해당 영역에서만 사용될 함수를 표현하는데 사용됩니다.     
주로 filter(), map(), reduce() 등의 함수와 함께 사용됩니다.

#### 기본 형식 
(lambda <인자> : <코드>) (전달인자)

In [6]:
def hap(x, y):
    return x + y
print(hap(10, 20))

(lambda x,y: x + y)(10, 20) # 위와 동일한 일을 함

30


30

일반적으로는 map(), filter() 등의 함수와 함께 쓰인다

In [12]:
a = [1,2,3,4]

list(map(lambda x: x ** 2, a))

[1, 4, 9, 16]

## CPython?

- Python is a language. 
- CPython은 C언어로 구현한 original Python implementation으로 기본적으로 python.org에서 다운받으면 이걸 사용하고 있는것.
- 마찬가지로 python언어로 구현한 것은 PyPy, Java로 구현한 것은 Jython 등이 있음.

즉 to distinguish the implementation of the language engine from the Python programming language itself.

## Singleton

- Class의 Instance가 생성될 때 Instance가 단 하나만 생성되게 하는 방법입니다.
- 때로는 클래스 내부 값이 여러 컨텍스트에서 값을 잃지 말고 지속적으로 유지해야할 때가 있기 때문에 Singleton은 필요한 기법입니다.
- 데이터베이스를 사용하는 등의 동작을 예로 들 수 있습니다.
- 출처: https://wonjayk.tistory.com/262 [배고파서 까먹고 만든 블로그]